In [91]:
import pandas as pd
import numpy as np
from pathlib import Path
from tabular.dl_logs import create_model_identifier, CALMIP_START_INDEX

%reload_ext autoreload
%autoreload 2

SCRIPT_PATH = Path('./scripts/calmip')
LOG_FILE = Path('./logs/calmip_logs.csv')

# args creation from logs

In [92]:
common_args = ["--task_name=long_term_forecast",
               "--is_training= 1",
               "--root_path = /tmpdir/arnaud/dataset/USC/",
               "--checkpoints = /tmpdir/arnaud/checkpoints/",
               "--results_path = /tmpdir/arnaud/results/",
               "--data_path = na",
               "--data = USC",
               "--des = Exp",
               "--batch_size = 32",
               "--train_epochs = 48",
               "--patience= 16",
               "--embed=fixed",
               "--consider_only_offense",
               "--inverse",
               "--itr= 3"]


In [93]:
ldf = pd.read_csv(LOG_FILE, index_col=0)
ldf = ldf[ldf.index >= CALMIP_START_INDEX[2]]
ldf

,task_name,is_training,model_id,model,data,root_path,data_path,features,target,freq,checkpoints,seq_len,label_len,pred_len,seasonal_patterns,inverse,mask_rate,anomaly_ratio,expand,d_conv,top_k,num_kernels,enc_in,dec_in,c_out,d_model,n_heads,e_layers,d_layers,d_ff,moving_avg,factor,distil,dropout,embed,activation,channel_independence,decomp_method,use_norm,down_sampling_layers,...,use_amp,use_gpu,gpu,use_multi_gpu,devices,p_hidden_dims,p_hidden_layers,use_dtw,augmentation_ratio,seed,jitter,scaling,permutation,randompermutation,magwarp,timewarp,windowslice,windowwarp,rotation,spawner,dtwwarp,shapedtwwarp,wdba,discdtw,discsdtw,extra_tag,patch_len,device_ids,epoch,fit_time,nb_params,nb_tr_params,nb_nontr_params,mae,mse,rmse,mape,mspe,fde,results_path
471,long_term_forecast,1,USC_16_16,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,/tmpdir/arnaud/checkpoints,16,4,16,Monthly,True,0.25,0.25,2,4,5,6,1,1,1,256,8,2,1,512,25,1,True,0.1,fixed,gelu,1,moving_avg,1,0,...,True,True,0,False,"0,1,2,3","[128, 128]",2,False,0,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,16,NaN,18.0,747.26,791809,791809,0,0.614937,0.878844,0.937467,inf,inf,1.597670,/tmpdir/arnaud/results
472,long_term_forecast,1,USC_24_24,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,/tmpdir/arnaud/checkpoints,24,8,24,Monthly,True,0.25,0.25,2,4,5,6,1,1,1,256,8,2,1,512,25,1,True,0.1,fixed,gelu,1,moving_avg,1,0,...,True,True,0,False,"0,1,2,3","[128, 128]",2,False,0,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,16,NaN,47.0,1061.27,791809,791809,0,0.913752,2.036071,1.426909,inf,inf,2.472808,/tmpdir/arnaud/results
473,long_term_forecast,1,USC_48_48,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,/tmpdir/arnaud/checkpoints,48,16,48,Monthly,True,0.25,0.25,2,4,5,6,1,1,1,256,8,2,1,512,25,1,True,0.1,fixed,gelu,1,moving_avg,1,0,...,True,True,0,False,"0,1,2,3","[128, 128]",2,False,0,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,16,NaN,40.0,1086.51,791809,791809,0,1.647258,7.078037,2.660458,inf,inf,4.432114,/tmpdir/arnaud/results
474,long_term_forecast,1,USC_32_32,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,/tmpdir/arnaud/checkpoints,32,8,32,Monthly,True,0.25,0.25,2,4,5,6,1,1,1,256,8,2,1,512,25,1,True,0.1,fixed,gelu,1,moving_avg,1,0,...,True,True,0,False,"0,1,2,3","[128, 128]",2,False,0,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,16,NaN,41.0,1127.05,791809,791809,0,1.195554,3.562750,1.887525,inf,inf,3.252716,/tmpdir/arnaud/results
475,long_term_forecast,1,USC_16_16,LSTM,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,/tmpdir/arnaud/checkpoints,16,4,16,Monthly,True,0.25,0.25,2,4,5,6,1,1,1,256,8,2,1,512,25,1,True,0.1,fixed,gelu,1,moving_avg,1,0,...,True,True,0,False,"0,1,2,3","[128, 128]",2,False,0,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,16,NaN,44.0,1069.61,791809,791809,0,0.607831,0.871429,0.933504,inf,inf,1.594972,/tmpdir/arnaud/results
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,long_term_forecast,1,USC_96_96,Transformer,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,/tmpdir/arnaud/checkpoints/,96,24,96,Monthly,True,0.25,0.25,2,4,5,6,1,1,1,256,8,2,1,512,25,1,True,0.1,fixed,gelu,1,moving_avg,1,0,...,True,True,0,False,"0,1,2,3","[128, 128]",2,False,0,2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,16,NaN,42.0,2248.03,4446721,1847809,2598912,2.517832,15.822498,3.977750,inf,inf,6.200127,/tmpdir/arnaud/results/
1005,long_term_forecast,1,USC_96_96,Transformer,USC,/tmpdir/arnaud/dataset/USC/,na,S,OT,h,/tmpdir/arnaud/checkp

In [94]:
ldf['exp_id'] = ldf.apply(lambda row : create_model_identifier(row), axis =1)
edf = ldf[(ldf["seq_len"]==16) & (ldf["features"]=="S") ][['exp_id',"fde","features",'fit_time','epoch']].groupby('exp_id').agg({'fde':'count','features':'count','fit_time':'sum', 'epoch' : 'mean'})
edf['fit_time'] = edf['fit_time'] / 3600
edf

,fde,features,fit_time,epoch
exp_id,,,,
DLinear_S_16-16_el2_dm512_nh8_dl1_ff2048,3,3,0.734414,38.000000
LSTM_S_16-16_el1_dm256_nh8_dl1_ff512,3,3,0.808700,46.000000
LSTM_S_16-16_el1_dm64_nh8_dl1_ff64,3,3,0.796625,45.333333
LSTM_S_16-16_el2_dm256_nh8_dl1_ff512,3,3,0.804517,36.666667
LSTM_S_16-16_el2_dm64_nh8_dl1_ff64,3,3,0.848978,46.000000
LSTM_S_16-16_el2_dm64_nh8_dl2_ff64,3,3,0.897356,46.666667
Nonstationary_Transformer_S_16-16_el2_dm128_nh8_dl1_ff128,3,3,4.479383,34.333333
Nonstationary_Transformer_S_16-16_el2_dm256_nh8_dl1_ff256,3,3,4.488947,37.333333
Nonstationary_Transformer_S_16-16_el2_dm64_nh4_dl1_ff64,3,3,4.476750,39.666667


In [95]:
time_limits = {
    'LSTM' : '02:00:00',
    'Transformer' :'08:00:00',
    'iTransformer' :'02:00:00',
    'PatchTST' :'06:00:00',
    'Nonstationary_Transformer' :'06:00:00',
    'TimesNet' :'06:00:00',
    'DLinear' :'02:00:00'    
}

In [96]:
tmp_df = ldf[(ldf["seq_len"]==16) & (ldf["features"]=="S") ].copy()

configs = []
config_df = tmp_df[['model','dropout','learning_rate','optimizer', "wd","e_layers",'d_model','n_heads', 'd_layers', 'd_ff']].copy()
configs = [v for v in config_df.drop_duplicates().T.to_dict().values()]
len(configs)    

27

In [106]:
configs_list = [[f"--{k}={v}" for k,v in config.items()]for config in configs]
configs_list

[['--model=LSTM',
  '--dropout=0.1',
  '--learning_rate=0.001',
  '--optimizer=adamw',
  '--wd=0.05',
  '--e_layers=2',
  '--d_model=256',
  '--n_heads=8',
  '--d_layers=1',
  '--d_ff=512'],
 ['--model=LSTM',
  '--dropout=0.1',
  '--learning_rate=0.001',
  '--optimizer=adamw',
  '--wd=0.05',
  '--e_layers=1',
  '--d_model=256',
  '--n_heads=8',
  '--d_layers=1',
  '--d_ff=512'],
 ['--model=LSTM',
  '--dropout=0.1',
  '--learning_rate=0.001',
  '--optimizer=adamw',
  '--wd=0.05',
  '--e_layers=1',
  '--d_model=64',
  '--n_heads=8',
  '--d_layers=1',
  '--d_ff=64'],
 ['--model=LSTM',
  '--dropout=0.1',
  '--learning_rate=0.001',
  '--optimizer=adamw',
  '--wd=0.05',
  '--e_layers=2',
  '--d_model=64',
  '--n_heads=8',
  '--d_layers=1',
  '--d_ff=64'],
 ['--model=LSTM',
  '--dropout=0.1',
  '--learning_rate=0.001',
  '--optimizer=adamw',
  '--wd=0.05',
  '--e_layers=2',
  '--d_model=64',
  '--n_heads=8',
  '--d_layers=2',
  '--d_ff=64'],
 ['--model=Transformer',
  '--dropout=0.1',
  '--le

In [98]:
features_set = {
   'S' : ["--features=S", "--enc_in=1", "--dec_in=1", "--c_out=1"],
   'MS' : ["--features=MS", "--enc_in=61", "--dec_in=1", "--c_out=1"]
}
time_horizons = [16,24,32,48,64,96]


In [99]:
exp_count = ldf.groupby('exp_id').count()['features']
exp_count

exp_id
DLinear_MS_24-24_el2_dm512_nh8_dl1_ff2048       3
DLinear_S_16-16_el2_dm512_nh8_dl1_ff2048        3
DLinear_S_24-24_el2_dm512_nh8_dl1_ff2048        3
DLinear_S_32-32_el2_dm512_nh8_dl1_ff2048        3
DLinear_S_48-48_el2_dm512_nh8_dl1_ff2048        3
                                               ..
iTransformer_S_64-64_el2_dm512_nh4_dl1_ff512    3
iTransformer_S_64-64_el3_dm256_nh4_dl1_ff256    3
iTransformer_S_64-64_el3_dm512_nh4_dl1_ff512    3
iTransformer_S_64-64_el4_dm256_nh8_dl1_ff256    3
iTransformer_S_64-64_el4_dm512_nh8_dl1_ff512    3
Name: features, Length: 181, dtype: int64

In [107]:
with open(SCRIPT_PATH / 'jobs_configs.txt', "w") as f:
    i = 0
    for th in time_horizons :
        th_args = [f"--model_id='USC_{th}_{th}'", f"--seq_len={th}", f"--pred_len={th}", f"--label_len={th//4}"]
        for feature, feature_args in features_set.items():
            for config_list, config_dict in zip(configs_list, configs) :
                args = th_args + feature_args + config_list + common_args
                identifier = f"{config_dict['model']}_{feature}_{th}-{th}_el{config_dict['e_layers']}_dm{config_dict['d_model']}_nh{config_dict['n_heads']}_dl{config_dict['d_layers']}_ff{config_dict['d_ff']}"
                str_args = f"{i}, {identifier}, {time_limits[config_dict['model']]}," + ",".join(args)
                if identifier not in exp_count or exp_count[identifier] != 3:
                    print(str_args)
                    i+=1
                    f.write(str_args.replace(' ',''))
                    f.write("\n")
        

0, Transformer_MS_16-16_el2_dm256_nh8_dl2_ff512, 08:00:00,--model_id='USC_16_16',--seq_len=16,--pred_len=16,--label_len=4,--features=MS,--enc_in=61,--dec_in=1,--c_out=1,--model=Transformer,--dropout=0.1,--learning_rate=0.0005,--optimizer=adamw,--wd=0.05,--e_layers=2,--d_model=256,--n_heads=8,--d_layers=2,--d_ff=512,--task_name=long_term_forecast,--is_training= 1,--root_path = /tmpdir/arnaud/dataset/USC/,--checkpoints = /tmpdir/arnaud/checkpoints/,--results_path = /tmpdir/arnaud/results/,--data_path = na,--data = USC,--des = Exp,--batch_size = 32,--train_epochs = 48,--patience= 16,--embed=fixed,--consider_only_offense,--inverse,--itr= 3
1, Transformer_MS_16-16_el3_dm256_nh8_dl2_ff512, 08:00:00,--model_id='USC_16_16',--seq_len=16,--pred_len=16,--label_len=4,--features=MS,--enc_in=61,--dec_in=1,--c_out=1,--model=Transformer,--dropout=0.1,--learning_rate=0.0005,--optimizer=adamw,--wd=0.05,--e_layers=3,--d_model=256,--n_heads=8,--d_layers=2,--d_ff=512,--task_name=long_term_forecast,--is_tra